<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#New" data-toc-modified-id="New-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>New</a></span></li><li><span><a href="#Old-IDF-TF-calc" data-toc-modified-id="Old-IDF-TF-calc-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Old IDF TF calc</a></span><ul class="toc-item"><li><span><a href="#Calc-IDF" data-toc-modified-id="Calc-IDF-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Calc IDF</a></span></li><li><span><a href="#Multiply-IDF-with-TF" data-toc-modified-id="Multiply-IDF-with-TF-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Multiply IDF with TF</a></span></li><li><span><a href="#PCA" data-toc-modified-id="PCA-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>PCA</a></span></li><li><span><a href="#Save" data-toc-modified-id="Save-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Save</a></span></li></ul></li></ul></div>

In [ ]:
import feather


from math import log

import numpy as np
import pandas as pd

from scipy import sparse

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path = '../data/processed/EDY/holdings_summary_s_s.feather'
summary = feather.read_dataframe(path)
summary.shape

In [ ]:
### Load the data files

path = '../data/processed/EDY/holdings_s_s.npz'
holdings = sparse.load_npz(path)

path = '../data/processed/EDY/holdings_b_b.npz'
holdings_b = sparse.load_npz(path)

print(holdings.shape)
print(holdings_b.shape)

In [ ]:
holdings = holdings
holdings_b = holdings_b

In [ ]:
port_number = holdings.shape[0]
port_number

### Load stock map

In [ ]:
path = '../data/interim/stock_map.npy'
stock_map = np.load(path)


path = '../data/interim/stock_map_t.npy'
stock_map_t = np.load(path) 

sm = stock_map.item().get('item1')

sm = {v: k for k, v in sm.items()}

In [ ]:
path = '../data/raw/holdings_co_info.feather'
stock_names = feather.read_dataframe(path)
print(stock_names.shape)

stock_names = stock_names.drop_duplicates()
stock_names.sample()

stock_names_dict = {row[0]: row[1] for row in stock_names.values}

In [ ]:
def get_stock_name(num):

In [ ]:
crsp_comp_key = sm.get(18834)
name = stock_names_dict.get(crsp_comp_key)

In [ ]:
stock_names_dict

# New 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
holdings_b

In [ ]:
test_holdings = holdings_b[:5_000,:]
test_holdings.shape

In [ ]:
# Get the text
text = ["Apple Google Facebook",
        "Apple Amazon Facebook"
       ]

# Vectorize the text
count = CountVectorizer(
#            min_df = 3, # A word has to appear at least x times to be included
#            max_df = 0.90, # Top (1-x)% most frequent words are removed since they propably are of limited value
#            ngram_range = (1,4) # What kind of ngrams to include. We include everything from 1-grams to 4-grams
            )

txt_count = count.fit_transform(text)


In [ ]:
# Fit LDA model
n_components = 3

lda = LatentDirichletAllocation(
    n_components = n_components, # How many topics are there -> hyper parameter
    random_state = 123, # random seed for reproducability
    learning_method = 'batch', # Evaluate all documents at once as opposed to the online method
    max_iter = 50, # After how many evaluation steps should the algo stop. Choose fewer for faster processing
    verbose = True, # Show each step
    n_jobs = 1,# Normally this parameter controlls how many parallel processes are used. However, on our MacBook it didn't work
)

X_topics = lda.fit_transform(test_holdings)
lda.components_.shape

In [ ]:
X_topics.shape

In [ ]:
topics_rounded = pd.DataFrame(X_topics.round())
topics_rounded = topics_rounded.idxmax(1)

In [ ]:
summary_joined = summary
summary_joined['topic'] = topics_rounded
a = summary_joined.loc[:1000,['cap_class']]
b = summary_joined.loc[:1000,['topic']]

pd.crosstab(summary_joined['cap_class'],summary_joined['topic'])

In [ ]:
# Print most important words of every topic
print("The most important words of our four topics are:")

n_top_words = 10
feature_names = np.arange(test_holdings.shape[1])
for topic_idx, topic in enumerate(lda.components_):
    print()
    print("Topic %d:" % (topic_idx + 1))
    print([feature_names[i]
    for i in topic.argsort()\
        [:-n_top_words - 1:-1]])


# Old IDF TF calc

### Round to int

In [ ]:
holdings.data = holdings.data.round().astype(int)

## Calc IDF

In [ ]:
col_sum = holdings[:,:].sum(0).T
plt.plot(col_sum, 'o')

In [ ]:
col_sum_b = holdings_b.sum(0).T
plt.plot(col_sum_b,'o')

In [ ]:
def IDF_calc(x,port_number):
    return 1 + log(port_number / (x))
IDF_calc = np.vectorize(IDF_calc)

IDFs = IDF_calc(col_sum_b,port_number)

In [ ]:
plt.plot(IDFs,'o')

## Multiply IDF with TF

In [ ]:
IDFs_matrix = np.tile(IDFs.T,(port_number,1))

In [ ]:
IDFs_matrix.shape

In [ ]:
holdings.shape

In [ ]:
holdings_transformed = holdings.multiply(IDFs_matrix).tocsr()

In [ ]:
holdings_transformed

## PCA

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
X = holdings_transformed.T
svd = TruncatedSVD(n_components=100)
svd.fit(X)  

In [ ]:
holdings_transformed = svd.components_

In [ ]:
print(svd.explained_variance_ratio_)  
print()
print(svd.explained_variance_ratio_.sum())  


In [ ]:
result_mat = pd.DataFrame({'PC1': svd.components_[1], 'PC2':svd.components_[0], 'Group': summary['crsp_obj_cd']})

sns.scatterplot('PC1','PC2', hue='Group', data = result_mat)

In [ ]:
holdings_transformed = pd.DataFrame(holdings_transformed).T

## Save

In [ ]:
### Load the data files
path = '../data/processed/EDY/holdings_s_lsa.npz'
feather.write_dataframe(holdings_transformed,path)